<a href="https://colab.research.google.com/github/siderbrand/PROYECTO-KAGGLE/blob/main/99_modelo_solucion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
elianabrand_cleaned_data_csv_path = kagglehub.dataset_download('elianabrand/cleaned-data-csv')

print('Data source import complete.')


In [ ]:
!pip install xgboost --quiet


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score

from xgboost import XGBClassifier

from tqdm.auto import tqdm
import random
import joblib


In [ ]:
df = pd.read_csv('/kaggle/input/cleaned-data-csv/cleaned_data.csv')
print("Dimensiones:", df.shape)
df.head()


In [ ]:
X = df.drop('RENDIMIENTO_GLOBAL', axis=1)
y = df['RENDIMIENTO_GLOBAL']

print("Dimensiones X:", X.shape)
print("Dimensiones y:", y.shape)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("Train:", X_train.shape, "Test:", X_test.shape)


In [ ]:
model = XGBClassifier(
    objective='multi:softmax',
    num_class=4,
    tree_method='hist',
    device='cuda',
    eval_metric='mlogloss'
)




In [ ]:
param_dist = {
    'n_estimators': [200, 400, 600],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [4, 6, 8, 10],
    'subsample': [0.7, 0.8, 1.0],
    'colsample_bytree': [0.7, 0.8, 1.0],
    'gamma': [0, 1, 3],
    'reg_lambda': [1, 5, 10]
}


In [ ]:
def random_search_xgb(model, param_dist, X_train, y_train, n_iter=20):
    best_score = -1
    best_params = None
    keys = list(param_dist.keys())

    print("\nIniciando búsqueda aleatoria...\n")

    for _ in tqdm(range(n_iter), desc="Buscando mejores parámetros (XGBoost GPU)"):
        params = {k: random.choice(param_dist[k]) for k in keys}

        try:
            model.set_params(**params)
            model.fit(X_train, y_train)

            # Evaluación segura (sin predict y sin mover datos CPU → GPU)
            score = model.score(X_train, y_train)

            if score > best_score:
                best_score = score
                best_params = params

        except Exception as e:
            print("Error:", e)
            continue

    return best_params, best_score



In [ ]:
best_params, best_score = random_search_xgb(
    model,
    param_dist,
    X_train,
    y_train,
    n_iter=90
)


In [ ]:
print("Mejores parámetros encontrados:", best_params)
print("Mejor accuracy de entrenamiento:", best_score)


In [ ]:
# Aplicar los mejores parámetros
model.set_params(**best_params)

# Entrenar modelo final
model.fit(X_train, y_train)

print("Modelo final entrenado con los mejores parámetros.")


In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

y_pred = model.predict(X_test)

print("Accuracy en Test:", accuracy_score(y_test, y_pred))
print("\nReporte de Clasificación:")
print(classification_report(y_test, y_pred))
